## PyTorch Geometric for Graph-Based Molecular Property Prediction using MoleculeNet benchmark



> ### This notebook contains the full code that was used to support the article *PyTorch Geometric for Graph-Based Molecular Property Prediction using MoleculeNet benchmark* on Medium. You can find the article [here](https://medium.com/@nikopavl4/pytorch-geometric-for-graph-based-molecular-property-prediction-using-moleculenet-benchmark-41e36369d3c6).

In [36]:
# Install required packages. PyTorch Geometric & Rdkit
!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git
!pip install rdkit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/



### Load Dataset



In [37]:
import torch
from torch_geometric.datasets import MoleculeNet

dataset = MoleculeNet(root=".", name="Tox21")

print()
print(f'Dataset: {dataset}:')
print('====================')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')

data = dataset[0]  # Get the first graph object.

print()
print(data)
print('=============================================================')

# Gather some statistics about the first graph.
print(f'Number of nodes: {data.num_nodes}')
print(f'Number of edges: {data.num_edges}')
print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')
print(f'Has isolated nodes: {data.has_isolated_nodes()}')
print(f'Has self-loops: {data.has_self_loops()}')
print(f'Is undirected: {data.is_undirected()}')


Dataset: Tox21(7831):
Number of graphs: 7831
Number of features: 9
Number of classes: 12

Data(x=[16, 9], edge_index=[2, 34], edge_attr=[34, 3], smiles='CCOc1ccc2nc(S(N)(=O)=O)sc2c1', y=[1, 12])
Number of nodes: 16
Number of edges: 34
Average node degree: 2.12
Has isolated nodes: False
Has self-loops: False
Is undirected: True


In [38]:
# How dataset node features look like
print(data.x)

tensor([[ 6,  0,  4,  5,  3,  0,  4,  0,  0],
        [ 6,  0,  4,  5,  2,  0,  4,  0,  0],
        [ 8,  0,  2,  5,  0,  0,  3,  0,  0],
        [ 6,  0,  3,  5,  0,  0,  3,  1,  1],
        [ 6,  0,  3,  5,  1,  0,  3,  1,  1],
        [ 6,  0,  3,  5,  1,  0,  3,  1,  1],
        [ 6,  0,  3,  5,  0,  0,  3,  1,  1],
        [ 7,  0,  2,  5,  0,  0,  3,  1,  1],
        [ 6,  0,  3,  5,  0,  0,  3,  1,  1],
        [16,  0,  4,  5,  0,  0,  4,  0,  0],
        [ 7,  0,  3,  5,  2,  0,  4,  0,  0],
        [ 8,  0,  1,  5,  0,  0,  3,  0,  0],
        [ 8,  0,  1,  5,  0,  0,  3,  0,  0],
        [16,  0,  2,  5,  0,  0,  3,  1,  1],
        [ 6,  0,  3,  5,  0,  0,  3,  1,  1],
        [ 6,  0,  3,  5,  1,  0,  3,  1,  1]])


### Feature Engineering

Since the features provided by Moleculenet is discrete and of type long, we need to convert them to continuous embeddings. There many ways to do this, we will simply use the Embedding module provided by PyTorch in an AtomEncoder.

In [39]:
class AtomEncoder(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(AtomEncoder, self).__init__()

        self.embeddings = torch.nn.ModuleList()

        for i in range(dataset.num_features):
            self.embeddings.append(torch.nn.Embedding(100, hidden_channels)) #num of embedding must be larger than the max num of nodes that any graph has

    def reset_parameters(self):
        for embedding in self.embeddings:
            embedding.reset_parameters()

    def forward(self, x):
        if x.dim() == 1:
            x = x.unsqueeze(1)

        out = 0
        for i in range(x.size(1)):
            out += self.embeddings[i](x[:, i])
        return out

In [40]:
# Create a toy sample of AtomEncoder
SimpleEncoder = AtomEncoder(hidden_channels=32)
# Print the embeddings weight matrix for the first feature
print(SimpleEncoder.embeddings[0].weight)
# The result after first graph passes the AtomEncoder
print(SimpleEncoder(data.x))


Parameter containing:
tensor([[-0.4098, -1.4001,  0.0499,  ..., -1.8789, -1.9344, -2.5145],
        [-1.2677,  0.4914, -1.9928,  ..., -0.3778, -0.8279,  0.8969],
        [-0.5105, -0.5357, -0.5866,  ..., -1.1122,  1.2513, -0.3600],
        ...,
        [-1.7138,  0.1144,  0.2715,  ..., -1.2818, -1.7340,  0.6490],
        [ 1.3216,  0.4623, -1.8051,  ..., -0.9364,  1.6566,  0.3482],
        [-0.1843, -1.3068, -0.0717,  ...,  1.5038,  0.6817, -0.6077]],
       requires_grad=True)
tensor([[ 1.3798,  6.2299, -2.2655,  3.0046, -1.8114,  2.9269, -1.6632, -1.2371,
          2.9854, -1.9348, -0.0806, -3.4514,  3.5337,  2.5041, -1.7789,  0.7235,
         -0.7574,  3.3493, -2.9603, -0.1788,  1.1836, -2.2927,  0.6847,  1.7603,
         -1.8460,  2.0293, -4.5602, -1.0920, -0.7553,  0.7818,  5.5969, -0.3150],
        [-0.3935,  2.5632,  0.2443,  4.1274, -2.2834,  5.2282, -0.8490,  0.5305,
          1.3624, -0.8127, -1.1887, -0.0944,  3.8778, -0.9077, -1.2529,  1.5761,
         -3.3738,  2.3159, -3.

### Initialize the model and split the dataset

In [41]:
from torch_geometric.loader import DataLoader
# We simply split the dataset by explicitly define a cutoff for training dataset (7831 graphs = 7000 training, 831 test)
train_dataset = dataset[:7000]
test_dataset = dataset[7000:]
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [42]:
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool as gap
import torch.nn.functional as F
from torch.nn import Linear
class GCN(torch.nn.Module):
    def __init__(self, hidden_channels, num_node_features, num_classes):
        super(GCN, self).__init__()
        torch.manual_seed(12345)
        self.emb = AtomEncoder(hidden_channels=32)
        self.conv1 = GCNConv(hidden_channels,hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.conv3 = GCNConv(hidden_channels, hidden_channels)
        self.lin = Linear(hidden_channels, num_classes)

    def forward(self,batch):
        x , edge_index, batch_size = batch.x, batch.edge_index, batch.batch
        x = self.emb(x)
        # 1. Obtain node embeddings 
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = self.conv3(x, edge_index)

        # 2. Readout layer
        x = gap(x, batch_size)  # [batch_size, hidden_channels]
        # 3. Apply a final classifier
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin(x)
        return x

In [43]:
import torch.optim as optim
model = GCN(hidden_channels=32, num_classes=dataset.num_classes, num_node_features=dataset.num_features)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.BCEWithLogitsLoss()

### The train function

In [44]:
def train(model, loader, optimizer):
    model.train()
    # For every batch in train loader
    for batch in loader:
        if batch.x.shape[0] == 1 or batch.batch[-1] == 0:
            pass
        else:
            pred = model(batch)
            optimizer.zero_grad()
            ## ignore nan targets (unlabeled) when computing training loss.
            is_labeled = batch.y == batch.y
            loss = criterion(pred.to(torch.float32)[is_labeled], batch.y.to(torch.float32)[is_labeled])
            loss.backward()
            optimizer.step()

### The test function

In [45]:
from sklearn.metrics import roc_auc_score
import numpy as np
def test(model, loader):
    model.eval()
    y_true = []
    y_pred = []
    # For every batch in test loader
    for batch in loader:

        if batch.x.shape[0] == 1:
            pass
        else:
            with torch.no_grad():
                pred = model(batch)

            y_true.append(batch.y.view(pred.shape))
            y_pred.append(pred)

    y_true = torch.cat(y_true, dim = 0).numpy()
    y_pred = torch.cat(y_pred, dim = 0).numpy()

    # Compute the ROC - AUC score and store as history
    rocauc_list = []

    for i in range(y_true.shape[1]):
        #AUC is only defined when there is at least one positive data.
        if np.sum(y_true[:,i] == 1) > 0 and np.sum(y_true[:,i] == 0) > 0:
            # ignore nan values
            is_labeled = y_true[:,i] == y_true[:,i]
            rocauc_list.append(roc_auc_score(y_true[is_labeled,i], y_pred[is_labeled,i]))

    if len(rocauc_list) == 0:
        raise RuntimeError('No positively labeled data available. Cannot compute ROC-AUC.')

    return {'rocauc': sum(rocauc_list)/len(rocauc_list)}

In [46]:
valid_curve = []
test_curve = []
train_curve = []
test_acc = []

for epoch in range(1, 20 + 1):
    print("Epoch {}".format(epoch))
    # Train model
    train(model, train_loader, optimizer)
    # Evaluate model
    train_perf = test(model, train_loader)
    test_perf = test(model, test_loader)

    print({'Train': train_perf, 'Test': test_perf})

Epoch 1
{'Train': {'rocauc': 0.6483830869621313}, 'Test': {'rocauc': 0.6248126749836761}}
Epoch 2
{'Train': {'rocauc': 0.707109619034378}, 'Test': {'rocauc': 0.6779452970037646}}
Epoch 3
{'Train': {'rocauc': 0.7235474943267931}, 'Test': {'rocauc': 0.6833641432311568}}
Epoch 4
{'Train': {'rocauc': 0.7305193622348054}, 'Test': {'rocauc': 0.6986799900101183}}
Epoch 5
{'Train': {'rocauc': 0.745452158802868}, 'Test': {'rocauc': 0.7034115130083652}}
Epoch 6
{'Train': {'rocauc': 0.7554822289478289}, 'Test': {'rocauc': 0.6918026810342117}}
Epoch 7
{'Train': {'rocauc': 0.7640886757334199}, 'Test': {'rocauc': 0.7076397261231001}}
Epoch 8
{'Train': {'rocauc': 0.7626600715314367}, 'Test': {'rocauc': 0.7149596571535298}}
Epoch 9
{'Train': {'rocauc': 0.7702799049391671}, 'Test': {'rocauc': 0.7112732741946953}}
Epoch 10
{'Train': {'rocauc': 0.7717259955431421}, 'Test': {'rocauc': 0.7294562737325125}}
Epoch 11
{'Train': {'rocauc': 0.7776402796403983}, 'Test': {'rocauc': 0.7268211563282007}}
Epoch 12
{